In [ ]:
import os
import torch
import json
import random
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

Set seed for reproducibility

In [2]:
def set_seed(seed=7):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(7)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [4]:
import pandas as pd

In [5]:
def load_liar_data(file_path):
    df = pd.read_csv(file_path, sep="\t", header=None)
    df = df[[1, 2]]  # label, statement
    df.columns = ["label", "text"]
    
    # Map to binary labels
    label_map = {
        "pants-fire": 0,
        "false": 0,
        "barely-true": 0,
        "half-true": 1,
        "mostly-true": 1,
        "true": 1
    }
    
    df = df[df["label"].isin(label_map.keys())]
    df["label"] = df["label"].map(label_map)
    
    return df.reset_index(drop=True)

In [6]:
train_path = "data/liar_dataset/train.tsv"
val_path = "data/liar_dataset/valid.tsv"
test_path = "data/liar_dataset/test.tsv"

In [7]:
train_df = load_liar_data(train_path)
val_df = load_liar_data(val_path)
test_df = load_liar_data(test_path)

In [8]:
print("Train size:", len(train_df))
print("Validation size:", len(val_df))
print("Test size:", len(test_df))

Train size: 10240
Validation size: 1284
Test size: 1267


In [9]:
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader

In [ ]:
class LIARDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = int(self.labels[idx])

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.float)
        }

In [11]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

train_dataset = LIARDataset(train_df["text"], train_df["label"], tokenizer)
val_dataset = LIARDataset(val_df["text"], val_df["label"], tokenizer)
test_dataset = LIARDataset(test_df["text"], test_df["label"], tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

In [12]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=1  # binary classification with sigmoid
)
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [15]:
from transformers import get_scheduler
from torch.optim import AdamW

In [16]:
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)

num_epochs = 3
num_training_steps = num_epochs * len(train_loader)

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)


In [17]:
from torch.nn import BCEWithLogitsLoss

In [18]:
loss_fn = BCEWithLogitsLoss()

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device).unsqueeze(1)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        loss = loss_fn(logits, labels)
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1} - Training Loss: {avg_loss:.4f}")


  0%|          | 0/640 [00:00<?, ?it/s]c:\Users\mca\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\module.py:1747: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
100%|██████████| 640/640 [03:57<00:00,  2.69it/s]


Epoch 1 - Training Loss: 0.6605


100%|██████████| 640/640 [03:55<00:00,  2.72it/s]


Epoch 2 - Training Loss: 0.5985


100%|██████████| 640/640 [03:55<00:00,  2.72it/s]

Epoch 3 - Training Loss: 0.4610


In [19]:
from sklearn.metrics import accuracy_score, f1_score

In [20]:
def evaluate_model(model, val_loader):
    model.eval()
    preds = []
    targets = []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device).unsqueeze(1)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            probabilities = torch.sigmoid(logits)
            predictions = (probabilities > 0.5).int()

            preds.extend(predictions.cpu().numpy())
            targets.extend(labels.cpu().numpy())

    acc = accuracy_score(targets, preds)
    f1 = f1_score(targets, preds)

    return acc, f1

In [21]:
val_acc, val_f1 = evaluate_model(model, val_loader)
print(f"Validation Accuracy: {val_acc:.4f} | F1 Score: {val_f1:.4f}")

Validation Accuracy: 0.6425 | F1 Score: 0.6797


In [22]:
torch.save(model.state_dict(), "models/bert_model_F1_0.6797.pt")